In [ ]:
import boto3
from sagemaker.inputs import TrainingInput
from datetime import datetime
import sagemaker
from sagemaker import image_uris
from sagemaker import Session
from sagemaker.workflow.pipeline_context import PipelineSession

# Define your S3 bucket and prefix
bucket = 'diabates'
prefix = 'pipeline'
region = 'ap-south-1'
input_source = f"s3://{bucket}/datasets/diabetes.csv"
train_path = f"s3://{bucket}/{prefix}/train"
test_path = f"s3://{bucket}/{prefix}/test"
val_path = f"s3://{bucket}/{prefix}/val"

# Generate a timestamp
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")  # Format: YYYYMMDD-HHMMSS
#model_output_uri
model_output_uri = f"s3://{bucket}/{prefix}/pipeline_run_version_{timestamp}"

train_input = TrainingInput(s3_data=train_path, content_type='text/csv')
evaluation_output_uri = evaluation_output_uri = f"s3://{bucket}/output/evaluation"

######instances-require for the pipeline###

processing_image_uri = image_uris.retrieve(framework='sklearn', region=region, version='1.0-1')
training_image_uri = image_uris.retrieve(framework='sklearn', region=region, version='1.0-1')
evaluation_image_uri = image_uris.retrieve(framework='sklearn', region=region, version='1.0-1')
instance_count = 1
instance_type = 'ml.m5.xlarge'


################# important for pipeline #####################
sagemaker_session = sagemaker.Session()
role = 'arn:aws:iam::590183717898:role/service-role/AmazonSageMaker-ExecutionRole-20240716T105741'  #sagemaker.get_execution_role()
pipeline_session = PipelineSession()

import sagemaker
from sagemaker.processing import (
    ProcessingInput,
    ProcessingOutput,
    ScriptProcessor,  # Use ScriptProcessor if you are using a script
)
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.workflow.pipeline import Pipeline

# Create a ScriptProcessor
sklearn_processor = ScriptProcessor(
    image_uri=processing_image_uri,
    role=role,
    instance_type=instance_type,
    instance_count=1, 
    base_job_name='diabetes_new',
    command=['python3'],
)

# Define processing step
processing_step = ProcessingStep(
    name='PreprocessingStep',
    processor=sklearn_processor,
    code='preprocess.py',  # Path to your preprocessing script
    inputs=[
        ProcessingInput(
            source=input_source, 
            destination="/opt/ml/processing/input",
            s3_input_mode="File",
            s3_data_distribution_type="ShardedByS3Key",
            
        )
    ],
    outputs=[
        ProcessingOutput(
            output_name="train_data", 
            source="/opt/ml/processing/output/train",
            destination=train_path,
            s3_upload_mode="EndOfJob",
        ),
        ProcessingOutput(
            output_name="test_data", 
            source="/opt/ml/processing/output/test",
            destination=test_path,
            s3_upload_mode="EndOfJob",
        ),
        ProcessingOutput(
            output_name="val_data", 
            source="/opt/ml/processing/output/validation",
            destination=val_path,
            s3_upload_mode="EndOfJob",
        ),
    ]
)

from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep
from sagemaker.workflow.pipeline import Pipeline
from datetime import datetime

# Define the input data for training
train_input = TrainingInput(s3_data=train_path, content_type='text/csv')

# Define the Estimator
estimator = Estimator(
    entry_point='train.py',  # Your training script
    image_uri=training_image_uri,
    py_version='py3',
    instance_type=instance_type,  # Specify the instance type
    instance_count=instance_count,  # Set the instance count
    role=role,
    output_path=model_output_uri,  # This should include the timestamp
    base_job_name='sklearn-diabetes',  # Update base job name if needed
    hyperparameters={
        'n_estimators': 50,
        'max_depth': 5
    }
)

# Define the training step
train_step = TrainingStep(
    name="TrainModel",
    estimator=estimator,
    inputs={
        "training": train_input,
    },
)

from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.processing import (
    ProcessingInput,
    ProcessingOutput,
    ScriptProcessor,
)
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.functions import JsonGet

##### train artifacts ######
#model_artifact_path = train_step.properties.ModelArtifacts.S3ModelArtifacts
model_artifact_path = train_step.properties.ModelArtifacts.S3ModelArtifacts

evaluation_report = PropertyFile(
    name="EvaluationReport",
    output_name="evaluation",
    path="evaluation.json"
)

# Initialize the ScriptProcessor
evaluation_processor = ScriptProcessor(
    image_uri=evaluation_image_uri,
    command=["python3"],
    instance_type=instance_type,
    instance_count=instance_count,
    role=role,  # Replace with your actual role ARN
    sagemaker_session=pipeline_session
)

# Define the processing step for evaluation
evaluation_step = ProcessingStep(
    name="EvaluateModel",
    processor=evaluation_processor,
    inputs=[
        ProcessingInput(source=model_artifact_path, destination="/opt/ml/processing/model"),
        ProcessingInput(source=test_path, destination="/opt/ml/processing/test"),
    ],
    outputs=[
        ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/output", destination=evaluation_output_uri),
    ],
    code="eval.py",
    property_files=[evaluation_report]
)

# Create the pipeline
'''pipeline = Pipeline(
    name='DiabetestrainingPipeline-4',
    steps=[train_step]  # Include the training step
)

# Create and start the pipeline
pipeline.create(role_arn=role)  # Creates the pipeline
pipeline.start()  # Starts the pipeline execution'''


####registerstep########
from sagemaker.workflow.conditions import ConditionGreaterThan
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.model import Model
from sagemaker.inputs import CreateModelInput
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.model_metrics import MetricsSource, ModelMetrics
from sagemaker.workflow.model_step import ModelStep

model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri=f"{evaluation_output_uri}/evaluation.json",
        content_type="application/json"
    )
)

container = image_uris.retrieve(region=aws_region, framework='xgboost', version='0.90-1')
model = Model(
    image_uri=container,
    model_data=model_artifact_path,
    role=role,  # Replace with your actual role ARN
    sagemaker_session=pipeline_session  # Attach the pipeline session here
)

register_args = model.register(
    content_types=["application/x-model"],
    response_types=["application/json"],
    inference_instances=["ml.m5.large"],
    transform_instances=["ml.m5.large"],
    model_package_group_name="diabates-main-structured",  # Specify the model group name here
    model_metrics=model_metrics  # Optional: Attach model metrics
)
step_register = ModelStep(name="diabates-main-structured", step_args=register_args)


# Define the condition to check accuracy
cond_gte = ConditionGreaterThan(
    left=JsonGet(
        step_name=evaluation_step.name,
        property_file=evaluation_report,
        json_path="accuracy"
    ),
    right=0.60
)

# Create a condition step
condition_step = ConditionStep(
    name="CheckAccuracy",
    conditions=[cond_gte],
    if_steps=[step_register],  # Register the model if accuracy > 60%
    else_steps=[]  # Do nothing if the accuracy is <= 60%
)

train_step.add_depends_on([processing_step])
evaluation_step.add_depends_on([train_step])

pipeline = Pipeline(
    name='DiabetesEndpointPipeline-2',
    steps=[processing_step, train_step, evaluation_step, condition_step]  # Include the training step
)

# Create and start the pipeline
pipeline.create(role_arn=role)  # Creates the pipeline
pipeline.start()  # Starts the pipeline execution
